In [9]:
import numpy as np
import pandas as pd

teams = pd.read_csv('teams.csv')
players = pd.read_csv('players.csv')

teams['Season'] = teams['Season'].astype(str).str.zfill(2)
players['Season'] = players['Season'].astype(str).str.zfill(2)
players.rename(columns={'Height (in)':'Height_(in)'}, inplace=True)

In [10]:
#Left-merging teams and players data sets on 'Player' 1/5 columns so the stats for players are added.

#Declaring various lists for merge:
#Right + left join on
default_columns = ['Season', 'Date', 'Home/Away', 'Game', 'Team', 'Season_Half']

#Explained later
update_order = ['Player_'+str(i) for i in range(1,6)]

#New dataset init
full = teams

#Sufixes to add for newly joined cols e.g. player 1-5
suffixes = ['_player'+str(i) for i in range(1,6)]

#Converting float whole number colums to ints
ints = ['FGM', 'FGA', '3PM', '3PA', 'FTM', 'FTA', 'OREB', 'DREB', 'REB',
        'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-', 'Height_(in)']

for player in suffixes:
    full = pd.merge(full, players, right_on=default_columns+['Player'],
                left_on=default_columns+[update_order[0]], how='left',
                suffixes=('_team', player))
    full = full.drop('Player', axis=1)
    full.rename(columns={'Height_(in)':'Height_(in)'+player,
                     'Min':'Min'+player,
                     '+/-':'+/-'+player,
                     'PTS':'PTS'+player}, inplace=True)
    
    #Moving remaining player names to the right so the structure of the data is:
        #Player 1 name, Player 1 stats, Player 2 name, Player 2 stats etc.
    del update_order[0]
    full = full[[col for col in list(full.columns) if col not in update_order]+update_order]
    
    #Columns that got _team prefix loose it in order to recieve it upon next iteration
    back = dict(zip(list(full.iloc[:, 14:31].columns), 
                    list(teams.iloc[:, 14:31].columns)))
    if player != '_player5':
        full.rename(columns=back, inplace=True)
    
    #DNP/INJ
    full = full.fillna(0)
    
    #Ints
    curr_ints = [i+player for i in ints]
    full[curr_ints] = full[curr_ints].astype(int)
    
full['Date'] = pd.to_datetime(full['Date'])

In [11]:
#Takes long to execute!

for season in full['Season'].unique():
    curr_data = full.loc[full['Season']==season]

    #Creating series with every team and transposing it
    team_list = pd.DataFrame(curr_data['Team'].unique(), columns=['Team'])
    team_list  = team_list.sort_values(by='Team').transpose()

    #Dates series
    dates = pd.DataFrame(curr_data['Date'].unique(), columns=['Date'])

    #Duplicationg teams to match the lenght of dates
    team_list = team_list.append([team_list]*(dates.shape[0] - 1), ignore_index=True)

    #Merging and stacking
    dummie = pd.concat([dates,team_list], axis=1)
    data = dummie.set_index('Date').stack().reset_index().drop('level_1', 
                                                            axis=1).rename(columns={0:'Team'})

    #Form columns
    not_avg =  ['Pl', 'He', '+/', 'Mi']
    avg_stats = [i for i in full.columns[14:] if i[:2] not in not_avg]
    cols = ['Season_Wins', 'PlayedHome', 'WonH1', 'WonQ1', 'Won']+avg_stats
    for col in cols:
        data[col] = np.nan
    
    #Filling columns
    for ix in range(data.shape[0]):
        curr_date = data['Date'][ix]
        curr_team = data.iloc[ix,1]
    
        #Entries for whole season stats
        entry = curr_data.loc[(curr_data['Date']<curr_date)&(curr_data['Team']==curr_team)]
        data.loc[ix, 'Season_Wins'] = entry['Won'].sum()
    
        #Entries for form stats (tail num for how many last games to monitor)
        entry = entry.tail(6)
        data.loc[ix, 'PlayedHome'] = entry.loc[entry['Home/Away']=='Home']['Home/Away'].shape[0]
    
        cumulative_ints = ['WonH1', 'WonQ1', 'Won']
        for col in cumulative_ints:
            data.loc[ix, col] = entry.loc[entry[col]==1][col].sum()
        
        averages = data.columns[7:]
        for col in averages:
            data.loc[ix, col] = entry[col].mean()
        
    if season == '96':
        forms = data
    else:
        forms = pd.concat([forms, data])
        
    print(f'Cicle done! Season: {season}')

Cicle done! Season: 96
Cicle done! Season: 97
Cicle done! Season: 98
Cicle done! Season: 99
Cicle done! Season: 00
Cicle done! Season: 01
Cicle done! Season: 02
Cicle done! Season: 03
Cicle done! Season: 04
Cicle done! Season: 05
Cicle done! Season: 06
Cicle done! Season: 07
Cicle done! Season: 08
Cicle done! Season: 09
Cicle done! Season: 10
Cicle done! Season: 11
Cicle done! Season: 12
Cicle done! Season: 13
Cicle done! Season: 14
Cicle done! Season: 15
Cicle done! Season: 16
Cicle done! Season: 17
Cicle done! Season: 18


In [12]:
forms = forms.fillna(0)
forms.to_csv('forms6.csv', index=False)

#full.to_csv('merged.csv', index=False)